In [44]:
import create_population_turf
import create_population_turf_newbie
import create_population_turf_nowin
import create_population_obstract
import create_population_dirt
import create_population_dirt_newbie
import create_population_dirt_nowin
import preprocessing
from feature_engineering import FeatureCreator
from feature_engineering_prediction import PredictionFeatureCreator

import prediction


import pandas as pd
import numpy as np


import condition_prediction


from train_lgbm_rank_niti_cross import Trainer_lightgbm_rank_niti_cv
from train_lightgbm_time import Trainer_lightgbm_time
from train_lgbm_time_cross import Trainer_lightgbm_time_cv
from train_lgbm_index_cross import Trainer_lightgbm_index_cv

from evaluation_lightgbm_time_kaiki import Evaluator_lightgbm_time_kaiki

from evaluation_shaft_time_kaiki_cross import Evaluator_lightgbm_time_kaiki_shaft

from evaluation_lgbm_rank_niti_cross import Evaluator_lightgbm_rank_niti_cross
from evaluation_shaft_rank_cross import Evaluator_lightgbm_rank_niti_shaft

from evaluation_lightgbm_index import Evaluator_lightgbm_index

from evaluation_shaft_index import Evaluator_lightgbm_index_shaft


%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
%autoreload

In [103]:
df = pd.read_csv("../data/03_train/evaluation_lightgbm_rank_niti_cv_dev_obstract.csv",sep="\t")
df

,race_id,horse_id,target,rank,tansho_odds,popularity,umaban,pred,pred_binary
0,201803030104,2009100335,0,2,16.5,8,9,0.084425,1
1,201803030104,2014100649,1,1,9.7,5,12,0.073689,1
2,201803030204,2012105949,0,8,22.0,6,4,0.084936,1
3,201803030204,2015110041,0,7,124.4,11,5,0.019865,0
4,201803030204,2012101246,0,6,159.9,13,2,0.023832,0
...,...,...,...,...,...,...,...,...,...
8322,202408060704,2020104922,0,3,3.1,1,9,0.024843,0
8323,202408060704,2021104924,0,4,13.9,5,3,0.039578,0
8324,202408060704,2017106222,0,5,175.3,11,1,0.013237,0
8325,202408060704,2017105602,0,6,3.5,2,5,0.024769,0


In [104]:
#期待値
df["expect_return"] = df["tansho_odds"] * df["pred"]
df

,race_id,horse_id,target,rank,tansho_odds,popularity,umaban,pred,pred_binary,expect_return
0,201803030104,2009100335,0,2,16.5,8,9,0.084425,1,1.393012
1,201803030104,2014100649,1,1,9.7,5,12,0.073689,1,0.714787
2,201803030204,2012105949,0,8,22.0,6,4,0.084936,1,1.868583
3,201803030204,2015110041,0,7,124.4,11,5,0.019865,0,2.471200
4,201803030204,2012101246,0,6,159.9,13,2,0.023832,0,3.810732
...,...,...,...,...,...,...,...,...,...,...
8322,202408060704,2020104922,0,3,3.1,1,9,0.024843,0,0.077012
8323,202408060704,2021104924,0,4,13.9,5,3,0.039578,0,0.550131
8324,202408060704,2017106222,0,5,175.3,11,1,0.013237,0,2.320399
8325,202408060704,2017105602,0,6,3.5,2,5,0.024769,0,0.086692


In [62]:
bet_df = df.query("expect_return > 1.0 and pred > 0.08")
bet_df 

,race_id,horse_id,target,rank,tansho_odds,popularity,umaban,pred,pred_binary,expect_return
4,201709040710,2013104956,0,3,7.7,4,10,0.151707,1,1.168142
41,201706040711,2012104678,0,10,13.5,7,4,0.142301,1,1.921062
55,201706040711,2012104008,0,3,5.4,3,9,0.242821,1,1.311235
58,201709040807,2013104488,0,12,15.7,8,12,0.125473,1,1.969930
60,201709040810,2012103929,0,3,20.1,8,5,0.108937,1,2.189640
...,...,...,...,...,...,...,...,...,...,...
89143,202407040110,2021100935,0,5,10.2,6,3,0.141294,1,1.441199
89145,202407040110,2021105926,0,3,4.7,2,4,0.277356,1,1.303575
89146,202407040110,2020103241,0,2,5.7,4,6,0.275453,1,1.570082
89147,202407040108,2021106118,0,15,13.2,6,3,0.165345,1,2.182559


In [63]:
#単勝オッズ（払い戻し）xターゲットは当たったか当たってないか、単勝
#的中率は調べてない、賭け方が1以上の場合にかけているから関係ない
bet_df = bet_df.copy()
bet_df.loc[:, "payoff"] = bet_df["target"] * bet_df["tansho_odds"]

bet_df

,race_id,horse_id,target,rank,tansho_odds,popularity,umaban,pred,pred_binary,expect_return,payoff
4,201709040710,2013104956,0,3,7.7,4,10,0.151707,1,1.168142,0.0
41,201706040711,2012104678,0,10,13.5,7,4,0.142301,1,1.921062,0.0
55,201706040711,2012104008,0,3,5.4,3,9,0.242821,1,1.311235,0.0
58,201709040807,2013104488,0,12,15.7,8,12,0.125473,1,1.969930,0.0
60,201709040810,2012103929,0,3,20.1,8,5,0.108937,1,2.189640,0.0
...,...,...,...,...,...,...,...,...,...,...,...
89143,202407040110,2021100935,0,5,10.2,6,3,0.141294,1,1.441199,0.0
89145,202407040110,2021105926,0,3,4.7,2,4,0.277356,1,1.303575,0.0
89146,202407040110,2020103241,0,2,5.7,4,6,0.275453,1,1.570082,0.0
89147,202407040108,2021106118,0,15,13.2,6,3,0.165345,1,2.182559,0.0


In [64]:
#払い戻しの合計
total_payoff = bet_df["payoff"].sum()
total_payoff

np.float64(9967.9)

In [65]:
#払い戻し、馬券購入回数、回収率
total_bet = len(bet_df)
return_rate = total_payoff / total_bet
total_payoff,total_bet,return_rate

(np.float64(9967.9), 12027, np.float64(0.8287935478506693))

In [66]:
#期待値がどれくらい上になるかを検証
np.linspace(0.8,10,10)

array([ 0.8       ,  1.82222222,  2.84444444,  3.86666667,  4.88888889,
        5.91111111,  6.93333333,  7.95555556,  8.97777778, 10.        ])

In [110]:
df["payoff"] = df["target"] * df["tansho_odds"]

In [147]:
from collections import defaultdict

result = defaultdict(list)
for exp in np.linspace(0.5,3,100):
    bet_df = df.query(f"expect_return > {exp} and pred > 0.07")
    total_payoff = bet_df["payoff"].sum()
    total_bet = len(bet_df)
    return_rate = total_payoff / total_bet
    result["expect_return"].append(exp)
    result["total_bet"].append(total_bet)    
    result["total_payoff"].append(total_payoff)
    result["return_rate"].append(return_rate)
result_df = pd.DataFrame(result)
result_df.head(100)

,expect_return,total_bet,total_payoff,return_rate
0,0.500000,2043,2002.1,0.979980
1,0.525253,1978,1952.0,0.986855
2,0.550505,1927,1912.1,0.992268
3,0.575758,1853,1849.9,0.998327
4,0.601010,1788,1783.7,0.997595
5,0.626263,1733,1706.5,0.984709
6,0.651515,1665,1637.6,0.983544
7,0.676768,1614,1606.5,0.995353
8,0.702020,1553,1560.2,1.004636
9,0.727273,1491,1513.6,1.015158


In [ ]:
from collections import defaultdict

result = defaultdict(list)
for exp in np.linspace(0.8,10,100):
    bet_df = df.query(f"expect_return > {exp}")
    total_payoff = bet_df["payoff"].sum()
    total_bet = len(bet_df)
    return_rate = total_payoff / total_bet
    result["expect_return"].append(exp)
    result["total_bet"].append(total_bet)    
    result["total_payoff"].append(total_payoff)
    result["return_rate"].append(return_rate)
result_df = pd.DataFrame(result)
result_df.head(100)

In [39]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

ModuleNotFoundError: No module named 'seaborn'

In [40]:
sns.lineplot(result_df,x="expect_return",y="return_rate")

NameError: name 'sns' is not defined

In [42]:
result_df.sort_values("return_rate",ascending = False).head()

,expect_return,total_bet,total_payoff,return_rate
2,0.824242,4445,3306.5,0.743870
3,0.836364,4385,3255.4,0.742395
1,0.812121,4491,3313.9,0.737898
0,0.800000,4547,3342.0,0.734990
10,0.921212,3988,2927.5,0.734077


In [66]:
#他の三連単系にも適応させる
import math
from pathlib import Path

import pandas as pd

DATA_DIR = Path("..", "data")
PREPROCESSED_DIR = DATA_DIR / "01_preprocessed"
TRAIN_DIR = DATA_DIR / "03_train"
OUTPUT_DIR = DATA_DIR / "04_evaluation"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

return_tables_filepath = PREPROCESSED_DIR / "return_tables.pickle"
train_dir = TRAIN_DIR
evaluation_filename = "evaluation_lightgbm_rank_niti_odds_removed.csv"
output_dir = OUTPUT_DIR
return_tables = pd.read_pickle(return_tables_filepath)
evaluation_df = pd.read_csv(train_dir / evaluation_filename, sep="\t")
output_dir = output_dir
evaluation_df

,race_id,horse_id,target,rank,tansho_odds,popularity,umaban,pred
0,202306030301,2020102181,1,1,1.9,1,12,0.227220
1,202306030301,2020101572,0,2,3.1,2,6,0.308430
2,202306030301,2020102135,0,3,23.4,6,10,0.082764
3,202306030301,2020103975,0,4,6.1,3,13,0.193020
4,202306030301,2020100381,0,5,54.8,7,2,0.019120
...,...,...,...,...,...,...,...,...
35150,202309050912,2017104873,0,12,172.3,13,11,0.008107
35151,202309050912,2019100653,0,13,94.7,12,15,0.013465
35152,202309050912,2017106240,0,14,75.0,10,7,0.029081
35153,202309050912,2018103205,0,15,368.3,15,16,0.009866


In [86]:
"""
sort_colで指定した列でソートし、上位n件のBOX馬券の的中率・回収率を
シミュレーションする関数。
"""
bet_df = (
    evaluation_df.sort_values("pred", ascending = False)
    .groupby("race_id")
    .head(3)
    .groupby("race_id")["umaban"]    
    # 払い戻しテーブルの馬番は文字列型なので合わせる
    .apply(lambda x: list(x.astype(str)))
    .reset_index()
         )
pred_df = (
    evaluation_df.sort_values("pred", ascending = False)
    .sort_values("pred", ascending = False)
    .groupby("race_id")
    .head(3)
    .groupby("race_id")["pred"]
    .apply(lambda x: list(x))
    .reset_index()
)
pred_df

,race_id,pred
0,202301010101,"[0.5077941893501549, 0.3208448203097965, 0.095..."
1,202301010102,"[0.3981832041800139, 0.2223949812396349, 0.092..."
2,202301010103,"[0.2373154014957641, 0.1784663199491415, 0.162..."
3,202301010104,"[0.1677502822473391, 0.1558535863536714, 0.124..."
4,202301010105,"[0.1507361826165545, 0.1198399329579728, 0.119..."
...,...,...
2587,202310030808,"[0.1865958325642172, 0.1520041532233221, 0.090..."
2588,202310030809,"[0.4517475478237358, 0.1288483883944573, 0.101..."
2589,202310030810,"[0.1904262978210795, 0.1258844150211567, 0.112..."
2590,202310030811,"[0.2223973151855648, 0.1828288977248285, 0.171..."


In [90]:
df = bet_df.merge(return_tables, on="race_id")
df = df.merge(pred_df, on="race_id")
df

,race_id,umaban,bet_type,win_umaban,return,pred
0,202301010101,"[5, 8, 1]",単勝,[5],120,"[0.5077941893501549, 0.3208448203097965, 0.095..."
1,202301010101,"[5, 8, 1]",複勝,[5],100,"[0.5077941893501549, 0.3208448203097965, 0.095..."
2,202301010101,"[5, 8, 1]",複勝,[8],110,"[0.5077941893501549, 0.3208448203097965, 0.095..."
3,202301010101,"[5, 8, 1]",複勝,[6],310,"[0.5077941893501549, 0.3208448203097965, 0.095..."
4,202301010101,"[5, 8, 1]",馬連,"[5, 8]",170,"[0.5077941893501549, 0.3208448203097965, 0.095..."
...,...,...,...,...,...,...
28495,202310030812,"[11, 6, 2]",ワイド,"[6, 8]",410,"[0.1658039725112311, 0.1232718706222467, 0.110..."
28496,202310030812,"[11, 6, 2]",ワイド,"[6, 11]",470,"[0.1658039725112311, 0.1232718706222467, 0.110..."
28497,202310030812,"[11, 6, 2]",馬単,"[8, 11]",1580,"[0.1658039725112311, 0.1232718706222467, 0.110..."
28498,202310030812,"[11, 6, 2]",三連複,"[6, 8, 11]",1590,"[0.1658039725112311, 0.1232718706222467, 0.110..."


,race_id,umaban,bet_type,win_umaban,return,pred,numeric_values
0,202301010101,"[5, 8, 1]",単勝,[5],120,"[0.5077941893501549, 0.3208448203097965, 0.095...","[0.5077941893501549, 0.3208448203097965, 0.095..."
1,202301010101,"[5, 8, 1]",複勝,[5],100,"[0.5077941893501549, 0.3208448203097965, 0.095...","[0.5077941893501549, 0.3208448203097965, 0.095..."
2,202301010101,"[5, 8, 1]",複勝,[8],110,"[0.5077941893501549, 0.3208448203097965, 0.095...","[0.5077941893501549, 0.3208448203097965, 0.095..."
3,202301010101,"[5, 8, 1]",複勝,[6],310,"[0.5077941893501549, 0.3208448203097965, 0.095...","[0.5077941893501549, 0.3208448203097965, 0.095..."
4,202301010101,"[5, 8, 1]",馬連,"[5, 8]",170,"[0.5077941893501549, 0.3208448203097965, 0.095...","[0.5077941893501549, 0.3208448203097965, 0.095..."
...,...,...,...,...,...,...,...
28495,202310030812,"[11, 6, 2]",ワイド,"[6, 8]",410,"[0.1658039725112311, 0.1232718706222467, 0.110...","[0.1658039725112311, 0.1232718706222467, 0.110..."
28496,202310030812,"[11, 6, 2]",ワイド,"[6, 11]",470,"[0.1658039725112311, 0.1232718706222467, 0.110...","[0.1658039725112311, 0.1232718706222467, 0.110..."
28497,202310030812,"[11, 6, 2]",馬単,"[8, 11]",1580,"[0.1658039725112311, 0.1232718706222467, 0.110...","[0.1658039725112311, 0.1232718706222467, 0.110..."
28498,202310030812,"[11, 6, 2]",三連複,"[6, 8, 11]",1590,"[0.1658039725112311, 0.1232718706222467, 0.110...","[0.1658039725112311, 0.1232718706222467, 0.110..."


In [ ]:
#期待値
df["expect_return"] = df["return"] * df["pred"]
df

In [94]:
#無理そうだし、期待値が1.78になるくらいの選択肢を評価するのは良さそう
#要するに手動で計算して、その値が出るように